In [1]:
from keras import backend as K
from keras.engine import Layer
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Lambda
from keras.layers import merge, concatenate

K.set_image_data_format('channels_last')

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import merge
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import SGD
from scipy.misc import imread
from scipy.misc import imresize

In [3]:

def crosschannelnormalization(alpha = 1e-4, k=2, beta=0.75, n=5,**kwargs):
    """
    This is the function used for cross channel normalization in the original
    Alexnet
    """
    def f(X):
        K.set_image_dim_ordering('th')
        if K.backend()=='tensorflow':
            b, ch, r, c = X.get_shape()
        else:
            b, ch, r, c = X.shape            
        half = n // 2
        square = K.square(X)
        extra_channels = K.spatial_2d_padding(K.permute_dimensions(square, (0,2,3,1))
                                              , ((0, 0), (half, half)))
        extra_channels = K.permute_dimensions(extra_channels, (0,3,1,2))
        scale = k
        for i in range(n):
            if K.backend()=='tensorflow':
                ch = int(ch)
            scale += alpha * extra_channels[:,i:i+ch,:,:]
        scale = scale ** beta
        return X / scale

    return Lambda(f, output_shape=lambda input_shape:input_shape,**kwargs)



def splittensor(axis=1, ratio_split=1, id_split=0,**kwargs):
    def f(X):
        if K.backend()=='tensorflow':
            div = int(X.get_shape()[axis]) // ratio_split
        else:
            div = X.shape[axis] // ratio_split

        if axis == 0:
            output =  X[id_split*div:(id_split+1)*div,:,:,:]
        elif axis == 1:
            output =  X[:, id_split*div:(id_split+1)*div, :, :]
        elif axis == 2:
            output = X[:,:,id_split*div:(id_split+1)*div,:]
        elif axis == 3:
            output = X[:,:,:,id_split*div:(id_split+1)*div]
        else:
            raise ValueError("This axis is not possible")

        return output

    def g(input_shape):
        output_shape=list(input_shape)
        output_shape[axis] = output_shape[axis] // ratio_split
        return tuple(output_shape)

    return Lambda(f,output_shape=lambda input_shape:g(input_shape),**kwargs)





In [4]:
if K.image_data_format()=='channels_first':
    inputs = Input(shape=(3, 227,227))
elif K.image_data_format()=='channels_last':
    inputs = Input(shape=(227,227,3))

In [5]:
conv_1 = Convolution2D(96, (11, 11), strides=(4, 4), activation='relu',
name='conv_1')(inputs)

In [6]:
conv_2 = MaxPooling2D((3, 3), strides=(2, 2))(conv_1)

In [7]:
conv_2 = crosschannelnormalization(name='convpool_1')(conv_2)

In [8]:
conv_2 = ZeroPadding2D((2, 2))(conv_2)

In [9]:
conv_2 = concatenate([
                       Convolution2D(128, (5, 5), activation='relu', name='conv_2_' + str(i + 1))(
                           splittensor(ratio_split=2, id_split=i)(conv_2)
                       ) for i in range(2)],  name='conv_2')
#mode='concat', concat_axis=1,

In [22]:
conv_3 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2)

In [11]:
conv_3 = crosschannelnormalization()(conv_3)

In [12]:
conv_3 = ZeroPadding2D((1, 1))(conv_3)

In [13]:
conv_3 = Convolution2D(384, (3, 3), activation='relu', name='conv_3')(conv_3)

In [14]:
conv_4 = ZeroPadding2D((1, 1))(conv_3)

In [15]:
conv_4 = concatenate([
                   Convolution2D(192, (3, 3), activation='relu', name='conv_4_' + str(i + 1))(
                       splittensor(ratio_split=2, id_split=i)(conv_4)
                   ) for i in range(2)], name='conv_4')


In [16]:
conv_5 = ZeroPadding2D((1, 1))(conv_4)

In [17]:
conv_5 = concatenate([
                   Convolution2D(128, (3, 3), activation='relu', name='conv_5_' + str(i + 1))(
                       splittensor(ratio_split=2, id_split=i)(conv_5)
                   ) for i in range(2)], name='conv_5')


In [18]:
dense_1 = MaxPooling2D((3, 3), strides=(2, 2), name='convpool_5')(conv_5)

In [19]:
dense_1 = Flatten(name='flatten')(dense_1)

In [20]:
dense_1 = Dense(4096, activation='relu', name='dense_1')(dense_1)

In [21]:
dense_2 = Dropout(0.5)(dense_1)

In [22]:
dense_2 = Dense(4096, activation='relu', name='dense_2')(dense_2)

In [23]:
dense_3 = Dropout(0.5)(dense_2)

In [24]:
dense_3 = Dense(1000, name='dense_3')(dense_3)

In [25]:
prediction = Activation('softmax', name='softmax')(dense_3)

In [28]:
model = Model(inputs=inputs, outputs=prediction)